In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm

In [2]:
#global variabls

directory_path = "/home/hao/AnacondaProjects/MLOntology/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [3]:
conceptLabelDict={}
errors=[]

def read_label(fname):
    conceptLabelDict={}
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptLabelDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)
    return conceptLabelDict

label_file_2017 = data_path + "ontClassLabels_july2017.txt"
conceptLabelDict_2017=read_label(label_file_2017)
label_file_2018= data_path + "ontClassLabels_jan2018.txt"
conceptLabelDict_2018=read_label(label_file_2018)

In [4]:
import json
from pprint import pprint

jsonFile = data_path + "2018newconcepts.json"

data = json.load(open(jsonFile))

# pprint(data)

# pprint(data['100841000119109']['Ancestors'][1])

In [5]:
def readFromJsonData(data):
    result_paired = []
    result_not_paired= []
    for key, value in data.items():
        if value['Parents']:
            for x in range(len(value['Parents'])):
                result_paired.append([key, value['Parents'][x], 1])
        if value['Siblings']:
            for x in range(len(value['Siblings'])):
                result_not_paired.append([key, value['Siblings'][x], 0])
        if value['Children']:
            for x in range(len(value['Children'])):
                result_not_paired.append([key, value['Children'][x], 0])
    return result_paired, result_not_paired
    
    

In [6]:
paired_list, not_paired_list = readFromJsonData(data)
pprint(paired_list[:20])
pprint(not_paired_list[:20])

[['737278006', '251019006', 1],
 ['293951000119107', '293931000119101', 1],
 ['293951000119107', '293961000119109', 1],
 ['186831000119104', '1131000119105', 1],
 ['186831000119104', '68345001', 1],
 ['186831000119104', '928000', 1],
 ['186831000119104', '363296001', 1],
 ['736990006', '735321000', 1],
 ['737070009', '431609005', 1],
 ['737070009', '736793003', 1],
 ['726116007', '301857004', 1],
 ['726116007', '8447006', 1],
 ['12247121000119101', '16302071000119103', 1],
 ['12247121000119101', '398199007', 1],
 ['12247121000119101', '428724006', 1],
 ['12247121000119101', '298149009', 1],
 ['734978006', '734977001', 1],
 ['350561000119109', '78261002', 1],
 ['350561000119109', '25082004', 1],
 ['350561000119109', '30721006', 1]]
[['737278006', '251026006', 0],
 ['737278006', '366204000', 0],
 ['737278006', '251020000', 0],
 ['737278006', '251033006', 0],
 ['737278006', '251021001', 0],
 ['186831000119104', '674361000119104', 0],
 ['736990006', '735326005', 0],
 ['736990006', '7353220

In [7]:
vector_model_file = vector_model_path + "model0"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

[('237267007', 0.6279136538505554),
 ('722912007', 0.6036121845245361),
 ('446466006', 0.592210054397583),
 ('722913002', 0.5905667543411255),
 ('67798003', 0.5715481042861938),
 ('253745002', 0.5682003498077393),
 ('10759661000119108', 0.567678689956665),
 ('277485007', 0.5653363466262817),
 ('177130000', 0.5645030736923218),
 ('312974005', 0.5602636337280273)]


In [8]:
import tensorflow as tf
 
sess=tf.Session()    
#First let's load meta graph and restore weights
cnn_model_file = cnn_model_path + 'model-noleaky.ckpt.meta'
saver = tf.train.import_meta_graph(cnn_model_file)
saver.restore(sess,tf.train.latest_checkpoint(cnn_model_path))

INFO:tensorflow:Restoring parameters from /home/hao/AnacondaProjects/MLOntology/cnnModel/model-noleaky.ckpt


In [9]:
graph = tf.get_default_graph()
x = graph.get_tensor_by_name("input_vector:0")
y = graph.get_tensor_by_name("class_label:0")
keep_prob = tf.placeholder(tf.float32)
predict= graph.get_tensor_by_name("predict:0")
correct_prediction= graph.get_tensor_by_name("correct_prediction:0")
accuracy= graph.get_tensor_by_name("accuracy:0")

In [ ]:
# correct_prediction = tf.equal(predict, tf.argmax(y,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"), name="accuracy")

In [15]:
feature_number = 1024

def getInferredVector(concept_id, conceptLabelDict, model):
    if concept_id in model.docvecs:
        concept_vector= model.docvecs[concept_id]
    else:
        concept_label = conceptLabelDict[concept_id]
        concept_vector= model.infer_vector(concept_label.split())
    return concept_vector

def getVectorFromModel(pair_list, conceptLabelDict, model):
    vector_list =[]
    label_list =[]
    for i, line in enumerate(pair_list):        
        a= getInferredVector(line[0], conceptLabelDict, model)
        b= getInferredVector(line[1], conceptLabelDict, model)
        c = np.array((a, b))
#         test_list.append(np.reshape(c, feature_number))
        vector_list.append(np.reshape(c, feature_number, order='F'))
        label_list.append(line[2])
    return vector_list, label_list

In [11]:
test_batch_size = 1024
pair_list = paired_list + not_paired_list
random.shuffle(pair_list)

In [16]:
print("%s iterations in total" % (len(pair_list)//test_batch_size))
for i in range(len(pair_list)//test_batch_size):
    list_batch = pair_list[ i*test_batch_size : min(i*test_batch_size +test_batch_size, len(pair_list))]
    vector_list_batch, label_list_batch  = getVectorFromModel(list_batch, conceptLabelDict_2018, vector_model)
    acc = sess.run(accuracy, feed_dict={x:vector_list_batch, y: np.eye(2)[label_list_batch], keep_prob:1})
    print("iteration %d Acc: %s" % (i,acc))

70 iterations in total
iteration 0 Acc: 0.113281
iteration 1 Acc: 0.135742
iteration 2 Acc: 0.146484
iteration 3 Acc: 0.131836
iteration 4 Acc: 0.128906
iteration 5 Acc: 0.137695
iteration 6 Acc: 0.133789
iteration 7 Acc: 0.142578
iteration 8 Acc: 0.126953
iteration 9 Acc: 0.111328
iteration 10 Acc: 0.130859
iteration 11 Acc: 0.137695
iteration 12 Acc: 0.132812
iteration 13 Acc: 0.129883
iteration 14 Acc: 0.130859
iteration 15 Acc: 0.123047
iteration 16 Acc: 0.125
iteration 17 Acc: 0.128906
iteration 18 Acc: 0.12207
iteration 19 Acc: 0.155273
iteration 20 Acc: 0.125977
iteration 21 Acc: 0.137695
iteration 22 Acc: 0.125
iteration 23 Acc: 0.107422
iteration 24 Acc: 0.116211
iteration 25 Acc: 0.125
iteration 26 Acc: 0.132812
iteration 27 Acc: 0.154297
iteration 28 Acc: 0.123047
iteration 29 Acc: 0.138672
iteration 30 Acc: 0.123047
iteration 31 Acc: 0.136719
iteration 32 Acc: 0.132812
iteration 33 Acc: 0.129883
iteration 34 Acc: 0.129883
iteration 35 Acc: 0.117188
iteration 36 Acc: 0.15136